<a href="https://colab.research.google.com/github/mshumer/gpt-prompt-engineer/blob/main/opus_to_haiku_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# opus-to-haiku - part of the `gpt-prompt-engineer` repo

This notebook gives you the ability to go from Claude Opus to Claude Haiku -- reducing costs massively while keeping quality high.

By Matt Shumer (https://twitter.com/mattshumer_)

Github repo: https://github.com/mshumer/gpt-prompt-engineer

In [ ]:
import requests
from dotenv import load_dotenv
import os

load_dotenv(".env")

ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")

In [ ]:
#@title Run this to prep the main functions

import json
import re

def generate_candidate_prompts(task, prompt_example, response_example):
    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }

    data = {
        "model": 'claude-3-opus-20240229',
        "max_tokens": 4000,
        "temperature": .5,
        "system": """<task>Given an example training sample, create seven additional samples for the same task that are even better. Each example should contain a <prompt> and a <response>.</task>

<rules>
1. Ensure the new examples are diverse and unique from one another.
2. They should all be perfect. If you make a mistake, this system won't work.
</rules>

Respond in this format:
<response_format>
<example_one>
<prompt>
PUT_PROMPT_HERE
</prompt>
<response>
PUT_RESPONSE_HERE
</response>
</example_one>

<example_two>
<prompt>
PUT_PROMPT_HERE
</prompt>
<response>
PUT_RESPONSE_HERE
</response>
</example_two>

...
</response_format>""",
        "messages": [
            {"role": "user", "content": f"""<training_task>{task}</training_task>

<prompt_example>
{prompt_example}
</prompt_example>

<response_example>
{response_example}
</response_example>"""},
        ]
    }


    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)

    response_text = response.json()['content'][0]['text']

    # Parse out the prompts and responses
    prompts_and_responses = []
    examples = re.findall(r'<example_\w+>(.*?)</example_\w+>', response_text, re.DOTALL)
    for example in examples:
        prompt = re.findall(r'<prompt>(.*?)</prompt>', example, re.DOTALL)[0].strip()
        response = re.findall(r'<response>(.*?)</response>', example, re.DOTALL)[0].strip()
        prompts_and_responses.append({'prompt': prompt, 'response': response})

    return prompts_and_responses

def generate_system_prompt(task, prompt_examples):
    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }

    data = {
        "model": 'claude-3-opus-20240229',
        "max_tokens": 1000,
        "temperature": .5,
        "system": """<your_role>Given a user-description of their <task> a set of prompt / response pairs (it'll be in JSON for easy reading) for the types of outputs we want to generate given inputs, write a fantastic system prompt that describes the task to be done perfectly.</your_role>

<rules>
1. Do this perfectly.
2. Respond only with the system prompt, and nothing else. No other text will be allowed.
</rules>

Respond in this format:
<system_prompt>
WRITE_SYSTEM_PROMPT_HERE
</system_prompt>""",
        "messages": [
            {"role": "user", "content": f"""<task>{task}</task>

<prompt_response_examples>
{str(prompt_examples)}
</prompt_response_examples>"""},
        ]
    }


    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)

    response_text = response.json()['content'][0]['text']

    # Parse out the prompt
    system_prompt = response_text.split('<system_prompt>')[1].split('</system_prompt>')[0].strip()

    return system_prompt

def test_haiku(generated_examples, prompt_example, system_prompt):
    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }

    messages = []

    for example in generated_examples:
      messages.append({"role": "user", "content": example['prompt']})
      messages.append({"role": "assistant", "content": example['response']})

    messages.append({"role": "user", "content": prompt_example.strip()})

    data = {
        "model": 'claude-3-haiku-20240307',
        "max_tokens": 2000,
        "temperature": .5,
        "system": system_prompt,
        "messages": messages,
    }


    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)

    response_text = response.json()['content'][0]['text']

    return response_text

def run_haiku_conversion_process(task, prompt_example, response_example):

    print('Generating the prompts / responses...')
    # Generate candidate prompts
    generated_examples = generate_candidate_prompts(task, prompt_example, response_example)

    print('Prompts / responses generated. Now generating system prompt...')

    # Generate the system prompt
    system_prompt = generate_system_prompt(task, generated_examples)

    print('System prompt generated:', system_prompt)


    print('\n\nTesting the new prompt on Haiku, using your input example...')
    # Test the generated examples and system prompt with the Haiku model
    haiku_response = test_haiku(generated_examples, prompt_example, system_prompt)

    print('Haiku responded with:')
    print(haiku_response)

    print('\n\n!! CHECK THE FILE DIRECTORY, THE PROMPT IS NOW SAVED THERE !!')

    # Create a dictionary with all the relevant information
    result = {
        "task": task,
        "initial_prompt_example": prompt_example,
        "initial_response_example": response_example,
        "generated_examples": generated_examples,
        "system_prompt": system_prompt,
        "haiku_response": haiku_response
    }

    # Save the Haiku prompt to a Python file
    with open("haiku_prompt.py", "w") as file:
        file.write('system_prompt = """' + system_prompt + '"""\n\n')

        file.write('messages = [\n')
        for example in generated_examples:
            file.write('    {"role": "user", "content": """' + example['prompt'] + '"""},\n')
            file.write('    {"role": "assistant", "content": """' + example['response'] + '"""},\n')

        file.write('    {"role": "user", "content": """' + prompt_example.strip() + '"""}\n')
        file.write(']\n')

    return result

## Fill in your task, prompt_example, and response_example here. Make sure you keep the quality really high here... this is the most important step!

In [ ]:
task = "refactoring complex code"

prompt_example = """def calculate_total(prices, tax, discount, shipping_fee, gift_wrap_fee, membership_discount):

    total = 0

    for i in range(len(prices)):

        total += prices[i]

    if membership_discount != 0:

        total = total - (total * (membership_discount / 100))

    if discount != 0:

        total = total - (total * (discount / 100))

    total = total + (total * (tax / 100))

    if total < 50:

        total += shipping_fee

    else:

        total += shipping_fee / 2

    if gift_wrap_fee != 0:

        total += gift_wrap_fee * len(prices)

    if total > 1000:

        total -= 50

    elif total > 500:

        total -= 25

    total = round(total, 2)

    if total < 0:

        total = 0

    return total"""

response_example = """def calculate_total(prices, tax_rate, discount_rate, shipping_fee, gift_wrap_fee, membership_discount_rate):

    def apply_percentage_discount(amount, percentage):

        return amount * (1 - percentage / 100)

    def calculate_shipping_fee(total):

        return shipping_fee if total < 50 else shipping_fee / 2

    def apply_tier_discount(total):

        if total > 1000:

            return total - 50

        elif total > 500:

            return total - 25

        return total

    subtotal = sum(prices)

    subtotal = apply_percentage_discount(subtotal, membership_discount_rate)

    subtotal = apply_percentage_discount(subtotal, discount_rate)



    total = subtotal * (1 + tax_rate / 100)

    total += calculate_shipping_fee(total)

    total += gift_wrap_fee * len(prices)



    total = apply_tier_discount(total)

    total = max(0, round(total, 2))



    return total"""

In [ ]:
task = "refactoring complex code"

prompt_example = """

You are an AI assistant that classifies a candidate's suitability for multiple job openings based on their CV. Your input will include:
- The candidate's CV, delimited by #### characters 
- One or more job IDs, each delimited by <> characters
- The corresponding job description for each job ID, delimited by ---- characters

For each job ID provided, analyze the candidate's CV against the job description and requirements. Classify the candidate's suitability into one of five categories:
- Highly Suitable 
- Moderately Suitable
- Potentially Suitable
- Marginally Suitable
- Not Suitable

In addition to the classification, provide a brief explanation for why you chose that suitability level, considering factors like:
- The candidate's years of relevant experience 
- The match between the candidate's skills and qualifications and the job requirements
- Any transferable skills from related industries
- Gaps in required or preferred qualifications

Generate your output as a list of JSON objects, with each object containing three keys:
- id: The job ID, corresponding to the ones delimited by <> characters. Do not include the <> characters.
- suitability: The classified suitability level
- explanation: The brief explanation for the classification

Example input 1:
####
John Doe
Software Engineer
5+ years experience in full-stack web development
Proficient in JavaScript, Python, React, Node.js
B.S. in Computer Science
####
<ID:21167>
----
Senior Frontend Engineer
- 5+ years frontend development experience 
- Expert in React, Redux, TypeScript
- Experience with unit testing
----
<ID:21239>
----
Data Scientist
- Advanced degree in quantitative field
- 3+ years experience in data analysis and modeling
- Proficiency in R, SQL, data visualization tools
----

Example output 1:
[
  {
    "id": "21167",
    "suitability": "Moderately Suitable",
    "explanation": "The candidate has 5+ years of relevant web development experience and proficiency in React, but lacks mentioned expertise in Redux or TypeScript."
  },
  {
    "id": "21239",
    "suitability": "Not Suitable",
    "explanation": "The candidate lacks an advanced degree and experience in data science. Their skills are not transferable to this role."
  }
]

Example input 2:
"
####
Jane Smith 
Marketing Manager
7 years of experience in digital marketing
Expertise in content strategy, SEO, social media, email campaigns
Proven track record of increasing web traffic and conversion rates
B.A. in Communication 
####
<ID:21301>
----
Digital Marketing Specialist 
- 3+ years experience in digital marketing
- Knowledge of SEO, SEM, social media, email marketing 
- Experience with Google Analytics, AdWords
- Strong writing and communication skills
----
<ID:21354>
----
Product Marketing Manager
- 5+ years in product marketing 
- Experience developing go-to-market strategies
- Collaborates with product, sales, PR teams
- Analyzes market trends and customer needs
----
"

Example output 2:
"
[
  {
    "id": "21301", 
    "suitability": "Highly Suitable",
    "explanation": "The candidate has 7 years of directly relevant digital marketing experience and expertise in all the required areas like SEO, social media, and email marketing."
  },
  {
    "id": "21354",
    "suitability": "Potentially Suitable", 
    "explanation": "While the candidate has extensive marketing experience, it's unclear if they have specific experience in product marketing, go-to-market strategies, and cross-functional collaboration. They may have transferable skills but lack some preferred qualifications."
  }
]
"

Example input 3: 
"
####
Michael Johnson
Sales Associate 
2 years retail sales experience 
Consistently exceeded sales quotas by 20%+
Strong interpersonal and customer service skills
Pursuing B.S. in Business Administration 
####
<ID:21402>
----
Account Executive 
- 5+ years B2B sales experience
- Proven track record of growing accounts
- Excellent presentation and negotiation skills
- Knowledge of CRM systems like Salesforce
----
<ID:21458>
----
Retail Store Manager
- 3+ years retail management experience 
- Track record of increasing store sales
- Coaches and mentors sales associates
- Ensures excellent customer experience 
----
"

Example output 3:
"
[
  {
    "id": "21402",
    "suitability": "Not Suitable",
    "explanation": "The candidate only has 2 years of retail sales experience, lacking the 5+ years of B2B sales experience required. Their skills are not directly transferable to this Account Executive role."
  },
  {
    "id": "21458", 
    "suitability": "Moderately Suitable",
    "explanation": "The candidate has 2 years of relevant retail sales experience and has demonstrated the ability to exceed sales goals. However, they lack the preferred 3+ years of retail management experience and leadership skills."
  }
]

"""

response_example = """"""

### Now, let's run this system and get our new prompt! At the end, you'll see a new file pop up in the directory that contains everything you'll need to reduce your costs while keeping quality high w/ Haiku!

In [ ]:
result = run_haiku_conversion_process(task, prompt_example, response_example)